**Use of Grid Search with CatBoost Regressor on the Boston Housing data**

**AWS Configuration required**

Runremotely use your credentials stored in ~/.aws/credential

Make sure you have install aws-cli and it's configured

https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2.html

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
sys.path.insert(0, os.path.abspath('..'))
from runremotely.runremotely import runremotely

<IPython.core.display.Javascript object>

In [3]:
# Import the usual suspects
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
import catboost
import pandas as pd
import numpy as np

**Load and Split data**

In [4]:
data = pd.read_csv('data.csv')

y = np.log(data.SalePrice)
X = data.drop(['SalePrice', 'Id'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=.33)

print(X_train.shape)

(978, 246)


**Initialize the Random Forest and the Grid Search**

In [5]:
param_grid = {'learning_rate': np.arange(0.05, 0.5, 0.01),
    'depth': range(1,3,1),
    'l2_leaf_reg': np.arange(0.1,2.0,0.1)}

model = CatBoostRegressor()

In [6]:
@runremotely(globals(), instance="c5.9xlarge")
def execute():
    return model.grid_search(param_grid, X_train, y=y_train, cv=5,plot=False)

In [7]:
grid = execute()

Requirements files: /var/folders/ms/xwkpspgd3kn4y2cb_q4639zh0000gn/T/tmpak4tp0c7.reqs
Creating key pair [/var/folders/ms/xwkpspgd3kn4y2cb_q4639zh0000gn/T/tmp6_9gyr_1.tp]
Launching instance (c5.9xlarge)
EC2 instance created:
	Id: i-091d7cbfd5646f9b1
	Public IP address: 34.253.230.236
Transfering [/var/folders/ms/xwkpspgd3kn4y2cb_q4639zh0000gn/T/tmp5d308ur4.tp] to ec2
Transfering [../template.py] to ec2
Transfering [/var/folders/ms/xwkpspgd3kn4y2cb_q4639zh0000gn/T/tmpak4tp0c7.reqs] to ec2
Executing [sudo yum -y install python37]
Executing [curl -O https://bootstrap.pypa.io/get-pip.py]
Error:   % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1824k  100 1824k    0     0  37.1M      0 --:--:-- --:--:-- --:--:-- 37.1M

Executing [sudo  python3 get-pip.py --user]
Error:   WARNING: The scripts pip, pip3 and pip3.7 are installed in '/root/.local/bin' which is not on PATH.
  Consider a

112:	loss: 0.1314422	best: 0.1287805 (62)	total: 46.2s	remaining: 10m 53s
113:	loss: 0.1326260	best: 0.1287805 (62)	total: 46.6s	remaining: 10m 52s
114:	loss: 0.1328786	best: 0.1287805 (62)	total: 47s	remaining: 10m 52s
115:	loss: 0.1314205	best: 0.1287805 (62)	total: 47.4s	remaining: 10m 51s
116:	loss: 0.1340005	best: 0.1287805 (62)	total: 47.8s	remaining: 10m 51s
117:	loss: 0.1339907	best: 0.1287805 (62)	total: 48.3s	remaining: 10m 51s
118:	loss: 0.1345995	best: 0.1287805 (62)	total: 48.7s	remaining: 10m 50s
119:	loss: 0.1372673	best: 0.1287805 (62)	total: 49.1s	remaining: 10m 50s
120:	loss: 0.1349088	best: 0.1287805 (62)	total: 49.5s	remaining: 10m 49s
121:	loss: 0.1349505	best: 0.1287805 (62)	total: 49.9s	remaining: 10m 49s
122:	loss: 0.1373249	best: 0.1287805 (62)	total: 50.3s	remaining: 10m 48s
123:	loss: 0.1336757	best: 0.1287805 (62)	total: 50.7s	remaining: 10m 48s
124:	loss: 0.1332671	best: 0.1287805 (62)	total: 51.1s	remaining: 10m 48s
125:	loss: 0.1318557	best: 0.1287805 (62

223:	loss: 0.1407360	best: 0.1287805 (62)	total: 1m 31s	remaining: 10m 6s
224:	loss: 0.1338544	best: 0.1287805 (62)	total: 1m 31s	remaining: 10m 6s
225:	loss: 0.1379732	best: 0.1287805 (62)	total: 1m 32s	remaining: 10m 6s
226:	loss: 0.1402144	best: 0.1287805 (62)	total: 1m 32s	remaining: 10m 5s
227:	loss: 0.1398521	best: 0.1287805 (62)	total: 1m 33s	remaining: 10m 5s
228:	loss: 0.1380169	best: 0.1287805 (62)	total: 1m 33s	remaining: 10m 4s
229:	loss: 0.1379055	best: 0.1287805 (62)	total: 1m 33s	remaining: 10m 4s
230:	loss: 0.1356761	best: 0.1287805 (62)	total: 1m 34s	remaining: 10m 4s
231:	loss: 0.1358595	best: 0.1287805 (62)	total: 1m 34s	remaining: 10m 3s
232:	loss: 0.1334717	best: 0.1287805 (62)	total: 1m 35s	remaining: 10m 3s
233:	loss: 0.1384560	best: 0.1287805 (62)	total: 1m 35s	remaining: 10m 2s
234:	loss: 0.1359784	best: 0.1287805 (62)	total: 1m 35s	remaining: 10m 2s
235:	loss: 0.1378666	best: 0.1287805 (62)	total: 1m 36s	remaining: 10m 1s
236:	loss: 0.1345131	best: 0.1287805 (

335:	loss: 0.1318387	best: 0.1287805 (62)	total: 2m 17s	remaining: 9m 21s
336:	loss: 0.1331571	best: 0.1287805 (62)	total: 2m 17s	remaining: 9m 20s
337:	loss: 0.1325565	best: 0.1287805 (62)	total: 2m 18s	remaining: 9m 20s
338:	loss: 0.1303196	best: 0.1287805 (62)	total: 2m 18s	remaining: 9m 19s
339:	loss: 0.1321814	best: 0.1287805 (62)	total: 2m 18s	remaining: 9m 19s
340:	loss: 0.1318076	best: 0.1287805 (62)	total: 2m 19s	remaining: 9m 19s
341:	loss: 0.1292099	best: 0.1287805 (62)	total: 2m 19s	remaining: 9m 18s
342:	loss: 0.1322876	best: 0.1287805 (62)	total: 2m 20s	remaining: 9m 18s
343:	loss: 0.1339461	best: 0.1287805 (62)	total: 2m 20s	remaining: 9m 17s
344:	loss: 0.1350268	best: 0.1287805 (62)	total: 2m 20s	remaining: 9m 17s
345:	loss: 0.1346938	best: 0.1287805 (62)	total: 2m 21s	remaining: 9m 17s
346:	loss: 0.1357064	best: 0.1287805 (62)	total: 2m 21s	remaining: 9m 16s
347:	loss: 0.1365115	best: 0.1287805 (62)	total: 2m 22s	remaining: 9m 16s
348:	loss: 0.1327591	best: 0.1287805 (

447:	loss: 0.1336370	best: 0.1287805 (62)	total: 3m 2s	remaining: 8m 35s
448:	loss: 0.1383927	best: 0.1287805 (62)	total: 3m 3s	remaining: 8m 34s
449:	loss: 0.1331317	best: 0.1287805 (62)	total: 3m 3s	remaining: 8m 34s
450:	loss: 0.1389856	best: 0.1287805 (62)	total: 3m 4s	remaining: 8m 34s
451:	loss: 0.1410287	best: 0.1287805 (62)	total: 3m 4s	remaining: 8m 33s
452:	loss: 0.1414758	best: 0.1287805 (62)	total: 3m 4s	remaining: 8m 33s
453:	loss: 0.1402709	best: 0.1287805 (62)	total: 3m 5s	remaining: 8m 32s
454:	loss: 0.1380975	best: 0.1287805 (62)	total: 3m 5s	remaining: 8m 32s
455:	loss: 0.1351629	best: 0.1287805 (62)	total: 3m 6s	remaining: 8m 31s
456:	loss: 0.1366782	best: 0.1287805 (62)	total: 3m 6s	remaining: 8m 31s
457:	loss: 0.1371065	best: 0.1287805 (62)	total: 3m 6s	remaining: 8m 31s
458:	loss: 0.1400908	best: 0.1287805 (62)	total: 3m 7s	remaining: 8m 30s
459:	loss: 0.1358431	best: 0.1287805 (62)	total: 3m 7s	remaining: 8m 30s
460:	loss: 0.1365784	best: 0.1287805 (62)	total: 3m

...

1227:	loss: 0.1647373	best: 0.1287805 (62)	total: 10m 13s	remaining: 4m
1228:	loss: 0.1662374	best: 0.1287805 (62)	total: 10m 14s	remaining: 4m
1229:	loss: 0.1612482	best: 0.1287805 (62)	total: 10m 14s	remaining: 3m 59s
1230:	loss: 0.1506755	best: 0.1287805 (62)	total: 10m 15s	remaining: 3m 59s
1231:	loss: 0.1567888	best: 0.1287805 (62)	total: 10m 16s	remaining: 3m 59s
1232:	loss: 0.1627191	best: 0.1287805 (62)	total: 10m 16s	remaining: 3m 58s
1233:	loss: 0.1634646	best: 0.1287805 (62)	total: 10m 17s	remaining: 3m 58s
1234:	loss: 0.1577879	best: 0.1287805 (62)	total: 10m 18s	remaining: 3m 57s
1235:	loss: 0.1534733	best: 0.1287805 (62)	total: 10m 19s	remaining: 3m 57s
1236:	loss: 0.1559915	best: 0.1287805 (62)	total: 10m 19s	remaining: 3m 56s
1237:	loss: 0.1536930	best: 0.1287805 (62)	total: 10m 20s	remaining: 3m 56s
1238:	loss: 0.1531738	best: 0.1287805 (62)	total: 10m 21s	remaining: 3m 56s
1239:	loss: 0.1530092	best: 0.1287805 (62)	total: 10m 21s	remaining: 3m 55s
1240:	loss: 0.155823

1336:	loss: 0.1445836	best: 0.1287805 (62)	total: 11m 30s	remaining: 3m 12s
1337:	loss: 0.1626069	best: 0.1287805 (62)	total: 11m 31s	remaining: 3m 12s
1338:	loss: 0.1568574	best: 0.1287805 (62)	total: 11m 32s	remaining: 3m 11s
1339:	loss: 0.1404082	best: 0.1287805 (62)	total: 11m 33s	remaining: 3m 11s
1340:	loss: 0.1427286	best: 0.1287805 (62)	total: 11m 33s	remaining: 3m 10s
1341:	loss: 0.1560961	best: 0.1287805 (62)	total: 11m 34s	remaining: 3m 10s
1342:	loss: 0.1483985	best: 0.1287805 (62)	total: 11m 35s	remaining: 3m 9s
1343:	loss: 0.1428916	best: 0.1287805 (62)	total: 11m 35s	remaining: 3m 9s
1344:	loss: 0.1516169	best: 0.1287805 (62)	total: 11m 36s	remaining: 3m 9s
1345:	loss: 0.1499502	best: 0.1287805 (62)	total: 11m 37s	remaining: 3m 8s
1346:	loss: 0.1566923	best: 0.1287805 (62)	total: 11m 38s	remaining: 3m 8s
1347:	loss: 0.1508391	best: 0.1287805 (62)	total: 11m 38s	remaining: 3m 7s
1348:	loss: 0.1583451	best: 0.1287805 (62)	total: 11m 39s	remaining: 3m 7s
1349:	loss: 0.16249

1445:	loss: 0.1470883	best: 0.1287805 (62)	total: 12m 50s	remaining: 2m 20s
1446:	loss: 0.1492586	best: 0.1287805 (62)	total: 12m 50s	remaining: 2m 20s
1447:	loss: 0.1530027	best: 0.1287805 (62)	total: 12m 51s	remaining: 2m 19s
1448:	loss: 0.1515666	best: 0.1287805 (62)	total: 12m 52s	remaining: 2m 19s
1449:	loss: 0.1548970	best: 0.1287805 (62)	total: 12m 53s	remaining: 2m 18s
1450:	loss: 0.1521901	best: 0.1287805 (62)	total: 12m 53s	remaining: 2m 18s
1451:	loss: 0.1584293	best: 0.1287805 (62)	total: 12m 54s	remaining: 2m 17s
1452:	loss: 0.1584287	best: 0.1287805 (62)	total: 12m 55s	remaining: 2m 17s
1453:	loss: 0.1691444	best: 0.1287805 (62)	total: 12m 56s	remaining: 2m 16s
1454:	loss: 0.1560412	best: 0.1287805 (62)	total: 12m 56s	remaining: 2m 16s
1455:	loss: 0.1545740	best: 0.1287805 (62)	total: 12m 57s	remaining: 2m 15s
1456:	loss: 0.1548644	best: 0.1287805 (62)	total: 12m 58s	remaining: 2m 15s
1457:	loss: 0.1518429	best: 0.1287805 (62)	total: 12m 59s	remaining: 2m 14s
1458:	loss: 

1554:	loss: 0.1564394	best: 0.1287805 (62)	total: 14m 12s	remaining: 1m 25s
1555:	loss: 0.1529649	best: 0.1287805 (62)	total: 14m 13s	remaining: 1m 24s
1556:	loss: 0.1671644	best: 0.1287805 (62)	total: 14m 14s	remaining: 1m 23s
1557:	loss: 0.1556507	best: 0.1287805 (62)	total: 14m 15s	remaining: 1m 23s
1558:	loss: 0.1556122	best: 0.1287805 (62)	total: 14m 15s	remaining: 1m 22s
1559:	loss: 0.1472627	best: 0.1287805 (62)	total: 14m 16s	remaining: 1m 22s
1560:	loss: 0.1554428	best: 0.1287805 (62)	total: 14m 17s	remaining: 1m 21s
1561:	loss: 0.1502200	best: 0.1287805 (62)	total: 14m 17s	remaining: 1m 21s
1562:	loss: 0.1394128	best: 0.1287805 (62)	total: 14m 18s	remaining: 1m 20s
1563:	loss: 0.1446846	best: 0.1287805 (62)	total: 14m 19s	remaining: 1m 20s
1564:	loss: 0.1444965	best: 0.1287805 (62)	total: 14m 20s	remaining: 1m 19s
1565:	loss: 0.1347042	best: 0.1287805 (62)	total: 14m 20s	remaining: 1m 19s
1566:	loss: 0.1490488	best: 0.1287805 (62)	total: 14m 21s	remaining: 1m 18s
1567:	loss: 

1664:	loss: 0.1503206	best: 0.1287805 (62)	total: 15m 31s	remaining: 25.2s
1665:	loss: 0.1393113	best: 0.1287805 (62)	total: 15m 31s	remaining: 24.6s
1666:	loss: 0.1397913	best: 0.1287805 (62)	total: 15m 32s	remaining: 24.1s
1667:	loss: 0.1406745	best: 0.1287805 (62)	total: 15m 33s	remaining: 23.5s
1668:	loss: 0.1441622	best: 0.1287805 (62)	total: 15m 34s	remaining: 22.9s
1669:	loss: 0.1496519	best: 0.1287805 (62)	total: 15m 34s	remaining: 22.4s
1670:	loss: 0.1504025	best: 0.1287805 (62)	total: 15m 35s	remaining: 21.8s
1671:	loss: 0.1520550	best: 0.1287805 (62)	total: 15m 36s	remaining: 21.3s
1672:	loss: 0.1483510	best: 0.1287805 (62)	total: 15m 36s	remaining: 20.7s
1673:	loss: 0.1559058	best: 0.1287805 (62)	total: 15m 37s	remaining: 20.2s
1674:	loss: 0.1492116	best: 0.1287805 (62)	total: 15m 38s	remaining: 19.6s
1675:	loss: 0.1606882	best: 0.1287805 (62)	total: 15m 39s	remaining: 19s
1676:	loss: 0.1573835	best: 0.1287805 (62)	total: 15m 39s	remaining: 18.5s
1677:	loss: 0.1608236	best:

In [10]:
print(grid['params'])

{'depth': 1, 'learning_rate': 0.22000000000000003, 'l2_leaf_reg': 0.2}
